In [3]:
from bs4 import BeautifulSoup

def extract_mdna_section(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    soup = BeautifulSoup(content, 'html.parser')
    
    # Extract content starting from "ITEM 7."
    mda_start = soup.find(string=lambda text: 'ITEM 7.' in text)
    if not mda_start:
        return "MD&A section not found."
    
    # Capture the content until we hit the next section
    mda_content = []
    for sibling in mda_start.find_all_next(string=True):
        if "ITEM 8." in sibling:
            break
        mda_content.append(sibling.strip())
    
    return ' '.join(mda_content)

# Example usage
file_path = 'data/sec-edgar-filings/MSFT/10-K/0001193125-04-150689/primary-document.html'
mdna_text = extract_mdna_section(file_path)
print(mdna_text)



In [4]:
file_path = 'data/sec-edgar-filings/MSFT/10-K/0001193125-04-150689/primary-document.html'
mdna_text = extract_mdna_section(file_path)
print(mdna_text)


 FINANCIAL CONDITION AND RESULTS OF OPERATIONS    RESULTS OF OPERATIONS FOR 2002, 2003, AND 2004    Management’s Discussion and Analysis (MD&A) contains statements that are forward-looking.
These statements are based on current expectations and assumptions that are subject to risks and uncertainties. Actual results could differ materially because of factors discussed in “Issues and Uncertainties” and elsewhere in this report.    OVERVIEW    The following Management’s Discussion and Analysis (“MD&A”) is intended to help the
reader understand Microsoft Corporation. MD&A is provided as a supplement to, and should be read in conjunction with, our financial statements and the accompanying notes (“Notes”).  We generate revenues, income and cash flows by developing, manufacturing, licensing, and supporting a wide range of software products for many computing devices.
Our software products include operating systems for servers, personal computers (PCs), and intelligent devices; server applicat